## **Recolha e Pré-processamento**

In [3]:
from Bio import Entrez
import pandas as pd

Entrez.email = "conhecimentolinguagem@gmail.com"
term = '("disease"[MeSH Terms]) AND ("symptom"[Title/Abstract] OR "treatment"[Title/Abstract]) AND ("2020"[Date - Publication] : "2025"[Date - Publication])'


handle = Entrez.esearch(db="pubmed", term=term, retmax=100)
record = Entrez.read(handle)
ids = record["IdList"]

articles = []
for pmid in ids:
    fetch = Entrez.efetch(db="pubmed", id=pmid, rettype="abstract", retmode="text")
    text = fetch.read()
    articles.append({"pmid": pmid, "text": text})

df = pd.DataFrame(articles)

# Save the articles to a CSV file
df.to_csv("articles.csv", index=False)

## **Extração de Entidades**

- Criar um ambiente virtual novo
- pip install scapy==3.7.4
- pip install scispacy==0.5.1
- Download de "en_core_sci_sm" em https://allenai.github.io/scispacy/
- pip install "location"


In [ ]:
import spacy

df = pd.read_csv("articles.csv")

nlp = spacy.load("en_core_sci_sm") # carrega o modelo do scispaCy

doc = nlp(df.iloc[0]['text'])
for ent in doc.ents:
    print(ent.text, ent.label_)

Ugeskr Laeger ENTITY
Acute vestibular syndrome ENTITY
Agger-Nielsen HE(1 ENTITY
Ovesen T(2) ENTITY
Author ENTITY
information ENTITY
Syddansk ENTITY
Universitetshospital ENTITY
Esbjerg ENTITY
(2)Universitetsklinikken ENTITY
Smag ENTITY
Søvn ENTITY
Regionshospitalet ENTITY
Gødstrup ENTITY
Acute vestibular syndrome ENTITY
AVS ENTITY
systematic approach ENTITY
central ENTITY
peripheral causes ENTITY
conditions ENTITY
stroke ENTITY
vestibular 
neuritis ENTITY
head impulse ENTITY
nystagmus ENTITY
skew deviation ENTITY
HINTS+ ENTITY
strokes ENTITY
treatment ENTITY
peripheral ENTITY
AVS ENTITY
vestibular rehabilitation ENTITY
central causes ENTITY
targeted therapies ENTITY
effective ENTITY
diagnosis ENTITY
management 
 ENTITY
outcomes ENTITY
quality of life ENTITY
review ENTITY
Published ENTITY
DOI ENTITY
PMID ENTITY
Indexed ENTITY
MEDLINE ENTITY
